This notebook created for Coursera capstone project 

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [3]:
import os
os.chdir("C:\\Users\\admin\\Desktop\\Caps")

In [4]:
pwd

'C:\\Users\\admin\\Desktop\\Caps'

In [5]:
data_project = pd.read_csv('projects.csv')

In [6]:
data_project['date_posted'] = pd.to_datetime(data_project['date_posted'], format= '%Y-%m-%d')

In [7]:
data_project.loc[data_project['date_posted'] < pd.Timestamp(2014,1,1)].shape[0]

619326

In [ ]:
# Data_outcome has records or Projects submitted before 2014-01-01 - It is consider as Training set


In [8]:
data_outcome = pd.read_csv('outcomes.csv')

In [9]:
data_essay = pd.read_csv('essays.csv',nrows=100000)

#Sample Audit data Report - Projects.csv Dataset

In [9]:
dmiss=pd.DataFrame((data_project.isnull().sum()/len(data_project.isnull()))*100,columns=['Miss Perc'])
dmiss['Miss']=data_project.isnull().sum()
summary_project=data_project.describe(percentiles=[0.05,0.10,0.25,0.50,0.75,0.85,0.95,0.99])
summary_project=summary_project.T
newsummary=pd.concat([dmiss,summary_project],axis=1)
newsummary.fillna(0,inplace=True)

In [10]:
newsummary.to_csv('sample_audit_project.csv')

#Sample Audit data Report - Essays Dataset

In [22]:
emiss=pd.DataFrame((data_essay.isnull().sum()/len(data_essay.isnull()))*100,columns=['Miss Perc'])
emiss['Miss']=data_essay.isnull().sum()
summary_essay=data_essay.describe(percentiles=[0.05,0.10,0.25,0.50,0.75,0.85,0.95,0.99])
summary_essay=summary_essay.T
newsummary1=pd.concat([emiss,summary_essay],axis=1)
newsummary1.fillna(0,inplace=True)

In [23]:
newsummary.to_csv('sample_audit_essay.csv')

In [73]:
data_project.head(2)

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,resource_type,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
0,316ed8fb3b81402ff6ac8f721bb31192,42d43fa6f37314365d08692e08680973,c0e6ce89b244764085691a1b8e28cb81,6.362701e+10,36.576340,-119.608713,Selma,CA,93662,NaN,...,Books,highest poverty,Grades 6-8,30.0,555.81,653.89,32.0,f,f,2014-05-12
1,90de744e368a7e4883223ca49318ae30,864eb466462bf704bf7a16a585ef296a,d711e47810900c96f26a5d0be30c446d,4.837020e+11,32.911179,-96.723640,Dallas,TX,75243,urban,...,Books,highest poverty,Grades PreK-2,30.0,296.47,348.79,22.0,f,f,2014-05-12


In [40]:
data_outcome.head(3)

,projectid,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count
0,ffffc4f85b60efc5b52347df489d0238,f,NaN,f,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN
1,ffffac55ee02a49d1abc87ba6fc61135,f,f,t,t,f,t,f,f,57.0,0.0,7.0
2,ffff97ed93720407d70a2787475932b0,f,f,t,t,t,t,t,f,100.0,0.0,3.0


#Merge Project and Essay dataset by using key ProjectID

In [10]:
data1= pd.merge(data_project,data_essay,on='projectid',how='inner')

In [11]:
data1.loc[data1['date_posted'] >= pd.Timestamp(2014,1,1)]['date_posted'].shape[0]

6670

#Generate the TEST_data_set by extract Project submitted after 2014-1-1

In [12]:
test_data=data1[data1['date_posted'] >= pd.Timestamp(2014,1,1)]

#Generate the TRAIN_data_set by extract Project submitted before 2014-1-1

In [13]:
train_datb=data1[data1['date_posted'] < pd.Timestamp(2014,1,1)]

In [14]:
data_project['projectid'] = data_project['projectid'].astype('str') 

In [15]:
data_outcome['projectid'] = data_outcome['projectid'].astype('str') 

In [16]:
data_train = pd.merge(data_project,data_outcome,on='projectid',how='inner')

In [17]:
# outcome.csv has only projects prior to 2014-1-1 i.e) train records.
data_train[data_train['date_posted'] > pd.Timestamp(2014,1,1)].shape[0]

0

In [18]:
train_dat=data_train[data_train['date_posted'] <= pd.Timestamp(2014,1,1)]

#Feature selections and EDA

In [19]:
dataAll_train= pd.merge(train_datb,train_dat[['projectid','is_exciting']],on='projectid',how='inner')

In [20]:
dataAll_train.shape

(93330, 41)

In [20]:
dataAll_train.columns

Index(['projectid', 'teacher_acctid_x', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_district', 'school_county',
       'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted', 'teacher_acctid_y', 'title', 'short_description',
       'need_statement', 'essay', 'is_exciting'],
      dtype='object')

In [21]:
dataAll_train['primary_focus_area'].value_counts()

Literacy & Language    42123
Math & Science         22670
Music & The Arts        8472
Applied Learning        7124
Special Needs           5882
History & Civics        4531
Health & Sports         2524
Name: primary_focus_area, dtype: int64

In [21]:
def area_tovalue(x):
    if x == 'Literacy & Language':
        return 6
    elif x == 'Math & Science':
        return 5
    elif x == 'Applied Learning':
        return 4
    elif x == 'Special Needs':
        return 3
    elif x == 'History & Civics':
        return 2
    else:
        return 1

In [22]:
dataAll_train['primary_focus_area']=dataAll_train['primary_focus_area'].apply(area_tovalue)

In [23]:
test_data['primary_focus_area']=test_data['primary_focus_area'].apply(area_tovalue)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
dataAll_train['primary_focus_area'].value_counts()

6    42123
5    22670
1    11000
4     7124
3     5882
2     4531
Name: primary_focus_area, dtype: int64

In [24]:
def plevel_tovalue(x):
    if x == 'highest poverty':
        return 5
    elif x == 'high poverty':
        return 4
    elif x == 'moderate poverty':
        return 3
    elif x == 'low poverty':
        return 2
    else:
        return 1

In [27]:
dataAll_train['poverty_level'].value_counts()

highest poverty     53947
high poverty        24477
moderate poverty    12597
low poverty          2309
Name: poverty_level, dtype: int64

In [25]:
dataAll_train['poverty_level'] = dataAll_train['poverty_level'].apply(plevel_tovalue)

In [26]:
test_data['poverty_level'] =test_data['poverty_level'].apply(plevel_tovalue)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
dataAll_train['students_reached'].count()

93311

#Extract Train dataset with predictors i'e) X and Y 

In [27]:
d_trainX = dataAll_train[['projectid','primary_focus_area','poverty_level','essay','is_exciting']]

#Extract Test dataset with X features

In [28]:
d_testX = test_data[['projectid','primary_focus_area','poverty_level','essay']]

In [29]:
#check null and data type
d_trainX.isnull().sum()
#d_trainX.dtypes

projectid             0
primary_focus_area    0
poverty_level         0
essay                 0
is_exciting           0
dtype: int64

In [30]:
d_testX.isnull().sum()

projectid             0
primary_focus_area    0
poverty_level         0
essay                 0
dtype: int64

In [32]:
d_trainX['is_exciting'].value_counts()

f    87751
t     5579
Name: is_exciting, dtype: int64

In [31]:
def objtostr(x):
    if x == 'f':
        return 1
    else:
        return 0

In [32]:
d_trainX['is_exciting'] = d_trainX['is_exciting'].apply(objtostr)

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
d_trainX.dtypes

projectid             object
primary_focus_area     int64
poverty_level          int64
essay                 object
is_exciting            int64
dtype: object

In [33]:
essay_dt = d_trainX['essay']
essaydt_lst =essay_dt.tolist() 


In [34]:
essay_x = d_testX['essay']
essaydt_x =essay_x.tolist() 


#By Using TfidfVectorizer convert Text into word matrix 

In [54]:
from sklearn.feature_extraction import text
tf=text.TfidfVectorizer(essaydt_lst,stop_words='english',max_features=1000)

In [57]:
#del matrix_tfidf_tst,d2,d1,X,y

In [55]:
matrix_tfidf=tf.fit_transform(essaydt_lst)

In [58]:
matrix_tfidf_tst=tf.fit_transform(essaydt_x)

In [59]:
d2 = pd.DataFrame(matrix_tfidf.toarray(),columns=tf.get_feature_names())
d2.shape

(93330, 1000)

In [60]:
d2_tst = pd.DataFrame(matrix_tfidf_tst.toarray(),columns=tf.get_feature_names())
d2_tst.shape

(6670, 1000)

In [62]:
print(d2.isnull().sum())
print(d2_tst.isnull().sum())

10         0
100        0
12         0
15         0
1st        0
          ..
writing    0
written    0
year       0
years      0
young      0
Length: 1000, dtype: int64
10         0
100        0
12         0
15         0
1st        0
          ..
writing    0
written    0
year       0
years      0
young      0
Length: 1000, dtype: int64


In [63]:
#del d1
d1=d_trainX[['primary_focus_area','poverty_level']]

In [64]:
d1tst=d_testX[['primary_focus_area','poverty_level']]

In [68]:
X=pd.concat([d2,d1],axis=1)

In [70]:
#X.isnull().sum()

In [71]:
X_test=pd.concat([d2_tst,d1tst],axis=1)

In [72]:
d_trainX.shape
#print(trainAll.shape)
print(X.columns)

Index(['10', '100', '12', '15', '1st', '20', '21st', '24', '25', '2nd',
       ...
       'world', 'write', 'writers', 'writing', 'written', 'year', 'years',
       'young', 'primary_focus_area', 'poverty_level'],
      dtype='object', length=1002)


In [73]:
print(X_test.columns)

Index(['10', '100', '12', '15', '1st', '20', '21st', '24', '25', '2nd',
       ...
       'world', 'write', 'writers', 'writing', 'written', 'year', 'years',
       'young', 'primary_focus_area', 'poverty_level'],
      dtype='object', length=1002)


In [74]:
#del y
y=d_trainX['is_exciting']

In [76]:
y.isnull().sum()

0

In [79]:
import sklearn.model_selection as model_selection
trainX,testX,trainY,testY=model_selection.train_test_split(X,y,test_size=0.2,random_state=200)

## LogisticRegression Model

In [77]:
import sklearn.linear_model as linear_model
clf=linear_model.LogisticRegression()

In [78]:
mod=clf.fit(X,y)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [86]:
#check the roc score 
import sklearn.metrics as metrics
predicted=mod.predict_proba(testX)
metrics.roc_auc_score(testY,predicted[:,1])

0.6746836483574834

#Train the model by using only 0.8 percent of Train Data set 

In [82]:
modTtrain=clf.fit(trainX,trainY)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [84]:
#check the roc score 
import sklearn.metrics as metrics
predicted_test=modTtrain.predict_proba(testX)
metrics.roc_auc_score(testY,predicted_test[:,1])

0.6746836483574834

##Regularize the model by using Gridsearch with  5 fold CV & l2 penalty 

In [85]:
np.random.seed(300)
modReg=model_selection.GridSearchCV(clf,param_grid={'penalty':['l2'],'C':np.random.uniform(0,40,2)},cv=5)


In [89]:
grd_cv=modReg.fit(X,y)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [90]:
print(grd_cv.best_estimator_)
print(grd_cv.best_score_)

LogisticRegression(C=18.04490274800219, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
0.9397835636987034


In [91]:
grd_predicted_test=grd_cv.predict_proba(testX)
metrics.roc_auc_score(testY,grd_predicted_test[:,1])

0.7279469309397625

In [92]:
grd_cv.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_penalty', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [152]:
def res(results,n=2):
    lst=[]
    for i in range(0,n):
        n = np.flatnonzero(results['rank_test_score'] == i)
        for j in n:
            lst.append(results['split0_test_score'][j])
            lst.append(results['split1_test_score'][j])
            lst.append(results['split2_test_score'][j])
            lst.append(results['split3_test_score'][j])
            lst.append(results['split4_test_score'][j])
    return lst

##Fold, AUC

In [153]:
res=res(grd_cv.cv_results_,2)

In [154]:
res

[0.9383906568091718,
 0.9398907103825137,
 0.940158577092039,
 0.9402657237758492,
 0.940212150433944]

In [155]:
auc_score=pd.DataFrame({'Fold':np.arange(0,5),'Auc':res})

In [157]:
auc_score.to_csv('AucReport.csv')

In [140]:
predicted_test=grd_cv.predict_proba(X_test)

In [141]:


did=test_data['projectid']
result=pd.DataFrame(list(zip(predicted_test[:,1],did)),columns=["is_exciting","projectId"])
result.to_csv('result.csv')

#Using NMF 

In [69]:
import sklearn.decomposition as decomposition
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt

In [72]:
#nmf=decomposition.PCA(100)
nmf=decomposition.NMF(n_components=100)
#scalers=preprocessing.StandardScaler()
#X1=scalers.fit_transform(matrix_tfidf)
#pca.fit_transform(X1)
matrix_reduced_nmf=nmf.fit_transform(matrix_tfidf)

In [73]:
matrix_reduced_nmf.shape

(139972, 100)

In [81]:
d3 = pd.DataFrame(matrix_reduced_nmf.tolist())
X1=pd.concat([d3,d1],axis=1)

In [82]:
X1_train,X1_test,y1_train,y1_test=model_selection.train_test_split(X1,y,test_size=0.2,random_state=200)

In [83]:
clf=linear_model.LogisticRegression()
nmfmod=clf.fit(X1_train,y1_train)


In [84]:
metrics.roc_auc_score(y1_test,nmfmod.predict_proba(X1_test)[:,1])

0.5966952401469932